In [24]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
import mnist 
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import tree
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import homogeneity_score

In [19]:
#training data 
path = "https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data"
data = pd.read_csv(path, header = None)

#splitting 20% data into test and 80 % into train 
X_train, X_test, Y_train, Y_test = train_test_split(data.iloc[:,0:57], data.iloc[:,57], test_size=0.2, random_state=42)

((3680, 57), (3680,), (921, 57), (921,))

In [29]:
inputs = 57
hidden_layer = 20
output_layer = inputs

X = tf.placeholder(tf.float32, shape = [None, inputs]) #flattened shape

tf.set_random_seed(42)
W = {
    'encoder_h1': tf.Variable(tf.random_normal([inputs, hidden_layer])),
    'decoder_h1':tf.Variable(tf.random_normal([hidden_layer, output_layer]))
}

B = {
    'encoder_h1':tf.Variable(tf.random_normal([hidden_layer])),
    'decoder_h1':tf.Variable(tf.random_normal([output_layer]))
}

encoder = tf.nn.sigmoid(tf.matmul(X,W['encoder_h1']) +  B['encoder_h1'])

decoder = tf.nn.sigmoid(tf.matmul(encoder,W['decoder_h1'])  + B['decoder_h1'])


epochs = 10
batch_size = 100
learning_rate = 0.01

Y = X
loss = tf.reduce_mean(tf.pow(decoder - Y, 2))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

def get_batches(batch_size,x):
        indexes = list(range(x.shape[0]))
        random.shuffle(indexes)
        ind = indexes[:batch_size]
        return(x[ind])
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        train_c = 0                                                       #cost
        total_batch = int(X_train.shape[0]/batch_size)
        for i in range(total_batch):
            batch_x = get_batches(batch_size, np.array(X_train))
            _,train_c = sess.run([optimizer, loss], feed_dict = {X: batch_x})
            train_c += train_c/batch_size    
        if epoch % 2 == 0:
            print("Epoch:",epoch+1,"\t Average Train Eror:",train_c)  
            test_c = sess.run(loss,feed_dict = {X:np.array(X_test)})
            print("\n Test Error", test_c)    
    encoder_train, decoder_train = sess.run([encoder,decoder],feed_dict = {X:np.array(X_train)})
    encoder_test, decoder_test = sess.run([encoder,decoder],feed_dict = {X:np.array(X_test)})
    
#For Classification Task:
print("Logistic Regression")
loReg = LogisticRegression(penalty='l2',solver = 'lbfgs',multi_class='multinomial')
loReg.fit(encoder_train,Y_train)

score = loReg.score(encoder_train, Y_train)
print("The training accuracy is ",score)

score = loReg.score(encoder_test,Y_test)
print("The testing accuracy is ",score)

#Clustering task:
print("K-means Clustering")
cluster = KMeans(n_clusters=3)
cluster.fit(encoder_train)
print("Train purity score", homogeneity_score(Y_train, cluster.predict(encoder_train)))
print("Test purity score", homogeneity_score(Y_test, cluster.predict(encoder_test)))

Epoch: 1 	 Average Train Eror: 3564.5937890625

 Test Error 11645.432
Epoch: 3 	 Average Train Eror: 10218.362333984374

 Test Error 11645.388
Epoch: 5 	 Average Train Eror: 13361.203203125

 Test Error 11645.385
Epoch: 7 	 Average Train Eror: 4458.504448242187

 Test Error 11645.386
Epoch: 9 	 Average Train Eror: 6867.049672851563

 Test Error 11645.385
Logistic Regression
The training accuracy is  0.653804347826087
The testing accuracy is  0.6568946796959826
K-means Clustering
score-1873.114649772644
Train purity score 0.06504292234112254
Test purity score 0.1025766599792918


In [30]:
inputs = 57
hidden_layer = 20
output_layer = inputs

X = tf.placeholder(tf.float32, shape = [None, inputs]) #flattened shape

tf.set_random_seed(42)
W = {
    'encoder_h1': tf.Variable(tf.random_normal([inputs, hidden_layer])),
    'decoder_h1':tf.Variable(tf.random_normal([hidden_layer, output_layer]))
}

B = {
    'encoder_h1':tf.Variable(tf.random_normal([hidden_layer])),
    'decoder_h1':tf.Variable(tf.random_normal([output_layer]))
}

encoder = tf.nn.sigmoid(tf.matmul(X,W['encoder_h1']) +  B['encoder_h1'])

decoder = tf.nn.sigmoid(tf.matmul(encoder,W['decoder_h1'])  + B['decoder_h1'])


epochs = 20
batch_size = 100
learning_rate = 0.01

Y = X
loss = tf.reduce_mean(tf.pow(decoder - Y, 2))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

def get_batches(batch_size,x):
        indexes = list(range(x.shape[0]))
        random.shuffle(indexes)
        ind = indexes[:batch_size]
        return(x[ind])
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        train_c = 0                                                       #cost
        total_batch = int(X_train.shape[0]/batch_size)
        for i in range(total_batch):
            batch_x = get_batches(batch_size, np.array(X_train))
            _,train_c = sess.run([optimizer, loss], feed_dict = {X: batch_x})
            train_c += train_c/batch_size    
        if epoch % 2 == 0:
            print("Epoch:",epoch+1,"\t Average Train Eror:",train_c)  
            test_c = sess.run(loss,feed_dict = {X:np.array(X_test)})
            print("\n Test Error", test_c)    
    encoder_train, decoder_train = sess.run([encoder,decoder],feed_dict = {X:np.array(X_train)})
    encoder_test, decoder_test = sess.run([encoder,decoder],feed_dict = {X:np.array(X_test)})
    
#For Classification Task:
print("Logistic Regression")
loReg = LogisticRegression(penalty='l2',solver = 'lbfgs',multi_class='multinomial')
loReg.fit(encoder_train,Y_train)

score = loReg.score(encoder_train, Y_train)
print("The training accuracy is ",score)

score = loReg.score(encoder_test,Y_test)
print("The testing accuracy is ",score)

#Clustering task:
print("K-means Clustering")
cluster = KMeans(n_clusters=3)
cluster.fit(encoder_train)
print("Train purity score", homogeneity_score(Y_train, cluster.predict(encoder_train)))
print("Test purity score", homogeneity_score(Y_test, cluster.predict(encoder_test)))

Epoch: 1 	 Average Train Eror: 4886.806586914063

 Test Error 11644.824
Epoch: 3 	 Average Train Eror: 3251.2215625

 Test Error 11643.377
Epoch: 5 	 Average Train Eror: 24173.86669921875

 Test Error 11643.361
Epoch: 7 	 Average Train Eror: 3858.7836596679685

 Test Error 11643.355
Epoch: 9 	 Average Train Eror: 6501.110102539063

 Test Error 11643.352
Epoch: 11 	 Average Train Eror: 7727.328515625

 Test Error 11643.339
Epoch: 13 	 Average Train Eror: 39363.53484375

 Test Error 11643.334
Epoch: 15 	 Average Train Eror: 2429.161208496094

 Test Error 11643.329
Epoch: 17 	 Average Train Eror: 6129.628354492187

 Test Error 11643.327
Epoch: 19 	 Average Train Eror: 6047.740434570313

 Test Error 11643.319
Logistic Regression
The training accuracy is  0.8565217391304348
The testing accuracy is  0.8621064060803475
K-means Clustering
Train purity score 0.35956635421357147
Test purity score 0.3549492816596936


In [31]:
inputs = 57
hidden_layer = 50
output_layer = inputs

X = tf.placeholder(tf.float32, shape = [None, inputs]) #flattened shape

tf.set_random_seed(42)
W = {
    'encoder_h1': tf.Variable(tf.random_normal([inputs, hidden_layer])),
    'decoder_h1':tf.Variable(tf.random_normal([hidden_layer, output_layer]))
}

B = {
    'encoder_h1':tf.Variable(tf.random_normal([hidden_layer])),
    'decoder_h1':tf.Variable(tf.random_normal([output_layer]))
}

encoder = tf.nn.sigmoid(tf.matmul(X,W['encoder_h1']) +  B['encoder_h1'])

decoder = tf.nn.sigmoid(tf.matmul(encoder,W['decoder_h1'])  + B['decoder_h1'])


epochs = 10
batch_size = 100
learning_rate = 0.01

Y = X
loss = tf.reduce_mean(tf.pow(decoder - Y, 2))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

def get_batches(batch_size,x):
        indexes = list(range(x.shape[0]))
        random.shuffle(indexes)
        ind = indexes[:batch_size]
        return(x[ind])
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        train_c = 0                                                       #cost
        total_batch = int(X_train.shape[0]/batch_size)
        for i in range(total_batch):
            batch_x = get_batches(batch_size, np.array(X_train))
            _,train_c = sess.run([optimizer, loss], feed_dict = {X: batch_x})
            train_c += train_c/batch_size    
        if epoch % 2 == 0:
            print("Epoch:",epoch+1,"\t Average Train Eror:",train_c)  
            test_c = sess.run(loss,feed_dict = {X:np.array(X_test)})
            print("\n Test Error", test_c)    
    encoder_train, decoder_train = sess.run([encoder,decoder],feed_dict = {X:np.array(X_train)})
    encoder_test, decoder_test = sess.run([encoder,decoder],feed_dict = {X:np.array(X_test)})
    
#For Classification Task:
print("Logistic Regression")
loReg = LogisticRegression(penalty='l2',solver = 'lbfgs',multi_class='multinomial')
loReg.fit(encoder_train,Y_train)

score = loReg.score(encoder_train, Y_train)
print("The training accuracy is ",score)

score = loReg.score(encoder_test,Y_test)
print("The testing accuracy is ",score)

#Clustering task:
print("K-means Clustering")
cluster = KMeans(n_clusters=3)
cluster.fit(encoder_train)
print("Train purity score", homogeneity_score(Y_train, cluster.predict(encoder_train)))
print("Test purity score", homogeneity_score(Y_test, cluster.predict(encoder_test)))

Epoch: 1 	 Average Train Eror: 3510.832741699219

 Test Error 11643.386
Epoch: 3 	 Average Train Eror: 3375.0397705078126

 Test Error 11643.365
Epoch: 5 	 Average Train Eror: 3858.9619384765624

 Test Error 11643.351
Epoch: 7 	 Average Train Eror: 5217.865649414062

 Test Error 11643.337
Epoch: 9 	 Average Train Eror: 39158.5008984375

 Test Error 11643.331
Logistic Regression
The training accuracy is  0.8654891304347826
The testing accuracy is  0.8599348534201955
K-means Clustering
Train purity score 0.04208621402465878
Test purity score 0.05261939662489349


In [32]:
inputs = 57
hidden_layer = 100
output_layer = inputs

X = tf.placeholder(tf.float32, shape = [None, inputs]) #flattened shape

tf.set_random_seed(42)
W = {
    'encoder_h1': tf.Variable(tf.random_normal([inputs, hidden_layer])),
    'decoder_h1':tf.Variable(tf.random_normal([hidden_layer, output_layer]))
}

B = {
    'encoder_h1':tf.Variable(tf.random_normal([hidden_layer])),
    'decoder_h1':tf.Variable(tf.random_normal([output_layer]))
}

encoder = tf.nn.sigmoid(tf.matmul(X,W['encoder_h1']) +  B['encoder_h1'])

decoder = tf.nn.sigmoid(tf.matmul(encoder,W['decoder_h1'])  + B['decoder_h1'])


epochs = 10
batch_size = 100
learning_rate = 0.01

Y = X
loss = tf.reduce_mean(tf.pow(decoder - Y, 2))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

def get_batches(batch_size,x):
        indexes = list(range(x.shape[0]))
        random.shuffle(indexes)
        ind = indexes[:batch_size]
        return(x[ind])
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        train_c = 0                                                       #cost
        total_batch = int(X_train.shape[0]/batch_size)
        for i in range(total_batch):
            batch_x = get_batches(batch_size, np.array(X_train))
            _,train_c = sess.run([optimizer, loss], feed_dict = {X: batch_x})
            train_c += train_c/batch_size    
        if epoch % 2 == 0:
            print("Epoch:",epoch+1,"\t Average Train Eror:",train_c)  
            test_c = sess.run(loss,feed_dict = {X:np.array(X_test)})
            print("\n Test Error", test_c)    
    encoder_train, decoder_train = sess.run([encoder,decoder],feed_dict = {X:np.array(X_train)})
    encoder_test, decoder_test = sess.run([encoder,decoder],feed_dict = {X:np.array(X_test)})
    
#For Classification Task:
print("Logistic Regression")
loReg = LogisticRegression(penalty='l2',solver = 'lbfgs',multi_class='multinomial')
loReg.fit(encoder_train,Y_train)

score = loReg.score(encoder_train, Y_train)
print("The training accuracy is ",score)

score = loReg.score(encoder_test,Y_test)
print("The testing accuracy is ",score)

#Clustering task:
print("K-means Clustering")
cluster = KMeans(n_clusters=3)
cluster.fit(encoder_train)
print("Train purity score", homogeneity_score(Y_train, cluster.predict(encoder_train)))
print("Test purity score", homogeneity_score(Y_test, cluster.predict(encoder_test)))

Epoch: 1 	 Average Train Eror: 5466.925336914062

 Test Error 11643.386
Epoch: 3 	 Average Train Eror: 3624.3655200195312

 Test Error 11643.355
Epoch: 5 	 Average Train Eror: 6771.954819335938

 Test Error 11643.337
Epoch: 7 	 Average Train Eror: 6941.78326171875

 Test Error 11643.329
Epoch: 9 	 Average Train Eror: 6137.773452148437

 Test Error 11643.317
Logistic Regression
The training accuracy is  0.9008152173913043
The testing accuracy is  0.9033659066232356
K-means Clustering
Train purity score 0.025741860280900582
Test purity score 0.023556397428143545


In [33]:
inputs = 57
hidden_layer = 200
output_layer = inputs

X = tf.placeholder(tf.float32, shape = [None, inputs]) #flattened shape

tf.set_random_seed(42)
W = {
    'encoder_h1': tf.Variable(tf.random_normal([inputs, hidden_layer])),
    'decoder_h1':tf.Variable(tf.random_normal([hidden_layer, output_layer]))
}

B = {
    'encoder_h1':tf.Variable(tf.random_normal([hidden_layer])),
    'decoder_h1':tf.Variable(tf.random_normal([output_layer]))
}

encoder = tf.nn.sigmoid(tf.matmul(X,W['encoder_h1']) +  B['encoder_h1'])

decoder = tf.nn.sigmoid(tf.matmul(encoder,W['decoder_h1'])  + B['decoder_h1'])


epochs = 10
batch_size = 100
learning_rate = 0.01

Y = X
loss = tf.reduce_mean(tf.pow(decoder - Y, 2))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

def get_batches(batch_size,x):
        indexes = list(range(x.shape[0]))
        random.shuffle(indexes)
        ind = indexes[:batch_size]
        return(x[ind])
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        train_c = 0                                                       #cost
        total_batch = int(X_train.shape[0]/batch_size)
        for i in range(total_batch):
            batch_x = get_batches(batch_size, np.array(X_train))
            _,train_c = sess.run([optimizer, loss], feed_dict = {X: batch_x})
            train_c += train_c/batch_size    
        if epoch % 2 == 0:
            print("Epoch:",epoch+1,"\t Average Train Eror:",train_c)  
            test_c = sess.run(loss,feed_dict = {X:np.array(X_test)})
            print("\n Test Error", test_c)    
    encoder_train, decoder_train = sess.run([encoder,decoder],feed_dict = {X:np.array(X_train)})
    encoder_test, decoder_test = sess.run([encoder,decoder],feed_dict = {X:np.array(X_test)})
    
#For Classification Task:
print("Logistic Regression")
loReg = LogisticRegression(penalty='l2',solver = 'lbfgs',multi_class='multinomial')
loReg.fit(encoder_train,Y_train)

score = loReg.score(encoder_train, Y_train)
print("The training accuracy is ",score)

score = loReg.score(encoder_test,Y_test)
print("The testing accuracy is ",score)

#Clustering task:
print("K-means Clustering")
cluster = KMeans(n_clusters=3)
cluster.fit(encoder_train)
print("Train purity score", homogeneity_score(Y_train, cluster.predict(encoder_train)))
print("Test purity score", homogeneity_score(Y_test, cluster.predict(encoder_test)))

Epoch: 1 	 Average Train Eror: 2779.1762646484376

 Test Error 11643.3955
Epoch: 3 	 Average Train Eror: 2179.334157714844

 Test Error 11643.363
Epoch: 5 	 Average Train Eror: 3659.7325341796877

 Test Error 11643.347
Epoch: 7 	 Average Train Eror: 6998.779711914062

 Test Error 11643.332
Epoch: 9 	 Average Train Eror: 6514.037905273438

 Test Error 11643.324
Logistic Regression
The training accuracy is  0.9182065217391304
The testing accuracy is  0.9066232356134636
K-means Clustering
Train purity score 0.057766720715459914
Test purity score 0.07726340201213858
